In [1]:
import pickle
import numpy
import plotly
from collections import defaultdict
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

### Plotter Functions

In [2]:
def plotFinalPriceWithTXF(x,y,z,xtitle, ytitle, ztitle):
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4']
        
    txDictx = defaultdict(list)
    txDictz = defaultdict(list)
    
    for i in range(len(y)):
        txDictx[y[i]].append(x[i])
        txDictz[y[i]].append(z[i])

    data = []   
         
    cnt = 0          
    for key in txDictx.keys():
        
        x_d = txDictx[key]
        y_d = [key for i in range(len(txDictx[key]))]
        z_d = txDictz[key]

        temp = go.Scatter3d(x=x_d, 
                         y=y_d, 
                         z=z_d,
                         name = "tx_fee: " + str(y_d[0]),
                         line=dict(
                            color=colors[cnt % len(colors)],
                            width=1
                        ) 
                    )
        cnt+=1      
        data.append(temp)
    
    fig = go.Figure(data = data)
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
        showlegend=True,              
    )

    plotly.offline.iplot(fig)
    
def plotWithCDP(cdp_axis, txf_axis, dai_price_history_axis, xtitle, ytitle, ztitle):
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4']
    
    data = []
    cnt = 0
    
    # generate an axis with numbers 1 -> number of days
    days_axis = [i for i in range(1, len(dai_price_history_axis[0]) + 1)]
    
    for k in range(0, len(dai_price_history_axis)):
        x_axis = [cdp_axis[k] for i in range(0, len(days_axis))]
        y_axis = days_axis
        z_axis = dai_price_history_axis[k]
        
        temp = go.Scatter3d(x=x_axis, 
                            y=y_axis, 
                            z=z_axis,
                            name = "tx_fee: " + str(txf_axis[k]) + " cdp_rate: " + str(cdp_axis[k]),
                            line=dict(
                            color=colors[cnt % len(colors)],
                            width=1
                           ) 
                        )
        
        cnt+=1      
        data.append(temp)
    
    fig = go.Figure(data = data)
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
        showlegend=True,              
    )

    plotly.offline.iplot(fig)
    
def plotWithRun(cdp_axis, txf_axis, run_axis, dai_axis, xtitle, ytitle, ztitle):
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4']
        
    point_dict = defaultdict(dict)
    
    # some data wrangling
    for i in range(len(dai_axis)):
        try:
            a = point_dict[run_axis[i]][txf_axis[i]]
        except:
            point_dict[run_axis[i]][txf_axis[i]] = []
        finally:
            point_dict[run_axis[i]][txf_axis[i]].append((cdp_axis[i], dai_axis[i]))
    
    data = []
    cnt = 0
    
    # plotting
    for key in point_dict:
        for inner_key in point_dict[key]:            
            cdp_x = [i[0] for i in point_dict[key][inner_key]]
            txf_y = [inner_key for i in range(len(point_dict[key][inner_key]))]
            dai_z = [i[1] for i in point_dict[key][inner_key]]
        
            temp = go.Scatter3d(x=cdp_x, 
                             y=txf_y, 
                             z=dai_z,
                            name = "tx_fee: " + str(inner_key) + " run: " + str(key),
                            line=dict(
                                color=colors[cnt % len(colors)],
                                width=1
                            ) 
                        )
            cnt+=1      
            data.append(temp)
    
    fig = go.Figure(data = data)
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)   

def plotWithTXF(txf_axis, cdp_axis, dai_price_history_axis, xtitle, ytitle, ztitle):
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4']
    
    data = []
    cnt = 0
    
    # generate an axis with numbers 1 -> number of days
    days_axis = [i for i in range(1, len(dai_price_history_axis[0]) + 1)]
    
    for k in range(0, len(dai_price_history_axis)):
        x_axis = [txf_axis[k] for i in range(0, len(days_axis))]
        y_axis = days_axis
        z_axis = dai_price_history_axis[k]
        
        temp = go.Scatter3d(x=x_axis, 
                            y=y_axis, 
                            z=z_axis,
                            name = "cdp_rate: " + str(cdp_axis[k]) + " tx_fee: " + str(txf_axis[k]),
                            line=dict(
                            color=colors[cnt % len(colors)],
                            width=1
                           ) 
                        )
        
        cnt+=1      
        data.append(temp)
    
    fig = go.Figure(data = data)
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)      

In [3]:
def extract_run_and_plot_final_settling_price(filename, run_id):
    input_file = open(filename,'rb')
    cdp_axis, txf_axis, run_axis, dai_price_history_axis = pickle.load(input_file)
    input_file.close()
    
    # Choose a random run a plot
    cdp = []
    txf = []
    daip = []
    for i in range(len(run_axis)):
        if run_axis[i] == run_id:
            cdp.append(cdp_axis[i])
            txf.append(txf_axis[i])
            daip.append(dai_price_history_axis[i][-1])
    plotFinalPriceWithTXF(cdp, txf, daip, "CDP Rate", "Transaction Fee", "DAI Price") 
    
def extract_run_and_plot_price_history_with_txf(filename, run_id):
    input_file = open(filename,'rb')
    cdp_axis, txf_axis, run_axis, dai_price_history_axis = pickle.load(input_file)
    input_file.close()
    
    # Choose a random run a plot
    cdp = []
    txf = []
    daip = []
    for i in range(len(run_axis)):
        if run_axis[i] == run_id:
            cdp.append(cdp_axis[i])
            txf.append(txf_axis[i])
            daip.append(dai_price_history_axis[i])
    plotWithTXF(txf, cdp, daip, "Transaction Fee", "Days", "DAI Price")     
    

def extract_run_and_plot_price_history_with_cdp(filename, run_id):
    input_file = open(filename,'rb')
    cdp_axis, txf_axis, run_axis, dai_price_history_axis = pickle.load(input_file)
    input_file.close()
    
    # Choose a random run a plot
    cdp = []
    txf = []
    daip = []
    for i in range(len(run_axis)):
        if run_axis[i] == run_id:
            cdp.append(cdp_axis[i])
            txf.append(txf_axis[i])
            daip.append(dai_price_history_axis[i])
    plotWithCDP(cdp, txf, daip, "CDP Rate", "Days", "DAI Price") 
    
def plot_final_settling_price_all_runs(filename):
    input_file = open(filename,'rb')
    cdp_axis, txf_axis, run_axis, dai_price_history_axis = pickle.load(input_file)
    dai_axis = [dai_price_history[-1] for dai_price_history in dai_price_history_axis]
    input_file.close()
    
    plotWithRun(cdp_axis, txf_axis, run_axis, dai_axis, "CDP Rate", "Transaction Fee", "DAI Price")

### Plot final settling DAI Price with TXF and CDPRate

In [14]:
extract_run_and_plot_final_settling_price("test1/sim-summary.pickle", 2)

### Plot DAI Price History with CDP Rate

In [16]:
extract_run_and_plot_price_history_with_cdp("test1/sim-summary.pickle", 0)

### Plot DAI Price History with Tx Fee

In [8]:
extract_run_and_plot_price_history_with_txf("test1/sim-summary.pickle", 1)

### Checking impact of initial distribution on settling price

In [10]:
plot_final_settling_price_all_runs("test1/sim-summary.pickle")

### Done
- Initial Distribution does affect the settling price a lot, but doesn't change dependency on CDP Rate and TX Fee.
- Settling Price increases, as CDP Rate Increases.
- Settling Price decreases, as TX Fee Increases.
- DAI Price mimics ETH Price variation closely. Absolute change in DAI Price is minimal (order of cents)

### Todo
- Check if changing initial DAI Price in each simulation affects the settling price. Ideally it should not depend on initial DAI Price.
- Check how number of investors affects DAI Settling Price.